
# 🏏 Cricbuzz Live Stats Project - MySQL + Jupyter

This notebook connects to **MySQL**, creates a database `cricbuzz_db`, builds 12 tables, and inserts sample cricket data.

**Connection Details:**  
- Username: `root`  
- Password: `monikha_19`


In [1]:

!pip install mysql-connector-python


In [11]:
import requests
import pandas as pd

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/live"

headers = {
	"x-rapidapi-key": "2ff90e9efdmsh244344514e3dddbp129be4jsn36f9f82e8a77",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
response = requests.get(url, headers=headers)

print(response.json())
data=response.json()

{'typeMatches': [{'matchType': 'International', 'seriesMatches': [{'seriesAdWrapper': {'seriesId': 10801, 'seriesName': 'Switzerland tour of Guernsey, 2025', 'matches': [{'matchInfo': {'matchId': 133704, 'seriesId': 10801, 'seriesName': 'Switzerland tour of Guernsey, 2025', 'matchDesc': '1st T20I', 'matchFormat': 'T20', 'startDate': '1756548000000', 'endDate': '1756560600000', 'state': 'Toss', 'status': 'Switzerland opt to bowl', 'team1': {'teamId': 533, 'teamName': 'Guernsey', 'teamSName': 'GGY', 'imageId': 172584}, 'team2': {'teamId': 1062, 'teamName': 'Switzerland', 'teamSName': 'SUI', 'imageId': 247207}, 'venueInfo': {'id': 563, 'ground': 'King George V Sports Ground', 'city': 'Castel', 'timezone': '+01:00', 'latitude': '49.469872', 'longitude': '-2.576630'}, 'seriesStartDt': '1756425600000', 'seriesEndDt': '1756684800000', 'isTimeAnnounced': True, 'stateTitle': 'Toss'}}]}}, {'seriesAdWrapper': {'seriesId': 10642, 'seriesName': 'United Arab Emirates T20I Tri-Series 2025', 'matches'

In [3]:
data

{'typeMatches': [{'matchType': 'International',
   'seriesMatches': [{'seriesAdWrapper': {'seriesId': 9602,
      'seriesName': 'South Africa tour of Australia, 2025',
      'matches': [{'matchInfo': {'matchId': 117020,
         'seriesId': 9602,
         'seriesName': 'South Africa tour of Australia, 2025',
         'matchDesc': '3rd ODI',
         'matchFormat': 'ODI',
         'startDate': '1756009800000',
         'endDate': '1756038600000',
         'state': 'Complete',
         'status': 'Australia won by 276 runs',
         'team1': {'teamId': 4,
          'teamName': 'Australia',
          'teamSName': 'AUS',
          'imageId': 172117},
         'team2': {'teamId': 11,
          'teamName': 'South Africa',
          'teamSName': 'RSA',
          'imageId': 172126},
         'venueInfo': {'id': 439,
          'ground': 'Great Barrier Reef Arena',
          'city': 'Mackay',
          'timezone': '+10:00',
          'latitude': '-21.1586312',
          'longitude': '149.1867182

In [4]:

import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="monikha_19"
)
cursor = conn.cursor()
print("✅ Connected to MySQL Server")


✅ Connected to MySQL Server


In [5]:

cursor.execute("DROP DATABASE IF EXISTS cricbuzz_db")
cursor.execute("CREATE DATABASE cricbuzz_db")
cursor.execute("USE cricbuzz_db")
print("✅ Database 'cricbuzz_db' created and selected")


✅ Database 'cricbuzz_db' created and selected


In [6]:

tables = {}

tables['teams'] = """CREATE TABLE teams(
    team_id INT AUTO_INCREMENT PRIMARY KEY,
    team_name VARCHAR(100) NOT NULL,
    country VARCHAR(50)
)"""

tables['players'] = """CREATE TABLE players(
    player_id INT AUTO_INCREMENT PRIMARY KEY,
    player_name VARCHAR(100) NOT NULL,
    role VARCHAR(50),
    team_id INT,
    FOREIGN KEY(team_id) REFERENCES teams(team_id)
)"""

tables['venues'] = """CREATE TABLE venues(
    venue_id INT AUTO_INCREMENT PRIMARY KEY,
    venue_name VARCHAR(150),
    city VARCHAR(100),
    country VARCHAR(50)
)"""

tables['tournaments'] = """CREATE TABLE tournaments(
    tournament_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    year INT
)"""

tables['matches'] = """CREATE TABLE matches(
    match_id INT AUTO_INCREMENT PRIMARY KEY,
    tournament_id INT,
    team1_id INT,
    team2_id INT,
    venue_id INT,
    match_date DATE,
    result VARCHAR(100),
    FOREIGN KEY(tournament_id) REFERENCES tournaments(tournament_id),
    FOREIGN KEY(team1_id) REFERENCES teams(team_id),
    FOREIGN KEY(team2_id) REFERENCES teams(team_id),
    FOREIGN KEY(venue_id) REFERENCES venues(venue_id)
)"""

tables['innings'] = """CREATE TABLE innings(
    innings_id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    team_id INT,
    runs INT,
    wickets INT,
    overs FLOAT,
    FOREIGN KEY(match_id) REFERENCES matches(match_id),
    FOREIGN KEY(team_id) REFERENCES teams(team_id)
)"""

tables['batting_stats'] = """CREATE TABLE batting_stats(
    batting_id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    player_id INT,
    runs INT,
    balls INT,
    fours INT,
    sixes INT,
    strike_rate FLOAT,
    FOREIGN KEY(match_id) REFERENCES matches(match_id),
    FOREIGN KEY(player_id) REFERENCES players(player_id)
)"""

tables['bowling_stats'] = """CREATE TABLE bowling_stats(
    bowling_id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    player_id INT,
    overs FLOAT,
    maidens INT,
    runs_conceded INT,
    wickets INT,
    economy FLOAT,
    FOREIGN KEY(match_id) REFERENCES matches(match_id),
    FOREIGN KEY(player_id) REFERENCES players(player_id)
)"""

tables['fall_of_wickets'] = """CREATE TABLE fall_of_wickets(
    fow_id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    wicket_number INT,
    runs INT,
    over_no FLOAT,
    player_out VARCHAR(100),
    FOREIGN KEY(match_id) REFERENCES matches(match_id)
)"""

tables['extras'] = """CREATE TABLE extras(
    extra_id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    byes INT,
    leg_byes INT,
    wides INT,
    no_balls INT,
    FOREIGN KEY(match_id) REFERENCES matches(match_id)
)"""

tables['umpires'] = """CREATE TABLE umpires(
    umpire_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    country VARCHAR(50)
)"""

tables['commentary'] = """CREATE TABLE commentary(
    comm_id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    over_no FLOAT,
    ball_no INT,
    description VARCHAR(255),
    FOREIGN KEY(match_id) REFERENCES matches(match_id)
)"""

for name, ddl in tables.items():
    cursor.execute(ddl)
    print(f"✅ Created table {name}")


✅ Created table teams
✅ Created table players
✅ Created table venues
✅ Created table tournaments
✅ Created table matches
✅ Created table innings
✅ Created table batting_stats
✅ Created table bowling_stats
✅ Created table fall_of_wickets
✅ Created table extras
✅ Created table umpires
✅ Created table commentary


In [7]:

# Insert Teams
cursor.execute("INSERT INTO teams (team_name, country) VALUES ('India', 'India'), ('Australia', 'Australia'), ('England', 'England')")

# Insert Players
cursor.execute("INSERT INTO players (player_name, role, team_id) VALUES ('Virat Kohli', 'Batsman', 1), ('Rohit Sharma', 'Batsman', 1), ('Jasprit Bumrah', 'Bowler', 1), ('Steve Smith', 'Batsman', 2), ('Pat Cummins', 'Bowler', 2)")

# Insert Venues
cursor.execute("INSERT INTO venues (venue_name, city, country) VALUES ('Wankhede Stadium', 'Mumbai', 'India'), ('MCG', 'Melbourne', 'Australia')")

# Insert Tournaments
cursor.execute("INSERT INTO tournaments (name, year) VALUES ('ICC World Cup', 2023)")

# Insert Match
cursor.execute("INSERT INTO matches (tournament_id, team1_id, team2_id, venue_id, match_date, result) VALUES (1, 1, 2, 1, '2023-10-15', 'India won by 6 wickets')")

# Insert Innings
cursor.execute("INSERT INTO innings (match_id, team_id, runs, wickets, overs) VALUES (1, 2, 250, 10, 48.3), (1, 1, 251, 4, 47.2)")

# Insert Batting Stats
cursor.execute("INSERT INTO batting_stats (match_id, player_id, runs, balls, fours, sixes, strike_rate) VALUES (1, 1, 80, 95, 8, 2, 84.2), (1, 2, 100, 88, 12, 3, 113.6)")

# Insert Bowling Stats
cursor.execute("INSERT INTO bowling_stats (match_id, player_id, overs, maidens, runs_conceded, wickets, economy) VALUES (1, 3, 10, 1, 40, 3, 4.0), (1, 5, 9.2, 0, 55, 2, 5.9)")

# Insert Fall of Wickets
cursor.execute("INSERT INTO fall_of_wickets (match_id, wicket_number, runs, over_no, player_out) VALUES (1, 1, 30, 5.2, 'David Warner')")

# Insert Extras
cursor.execute("INSERT INTO extras (match_id, byes, leg_byes, wides, no_balls) VALUES (1, 2, 4, 5, 1)")

# Insert Umpires
cursor.execute("INSERT INTO umpires (name, country) VALUES ('Aleem Dar', 'Pakistan'), ('Nitin Menon', 'India')")

# Insert Commentary
cursor.execute("INSERT INTO commentary (match_id, over_no, ball_no, description) VALUES (1, 12.3, 3, 'Virat Kohli drives through cover for four!')")

conn.commit()
print("✅ Sample data inserted into all tables")


✅ Sample data inserted into all tables


In [8]:

cursor.execute("SELECT * FROM batting_stats")
for row in cursor.fetchall():
    print(row)


(1, 1, 1, 80, 95, 8, 2, 84.2)
(2, 1, 2, 100, 88, 12, 3, 113.6)


In [9]:

cursor.close()
conn.close()
print("✅ MySQL connection closed")



✅ MySQL connection closed


In [10]:
cursor.execute("USE cricbuzz_db;")


ProgrammingError: 2055: Cursor is not connected

In [ ]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="monikha_19",
    database="cricbuzz_db"   # 👈 add this
)


In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="monikha_19",
    database="cricbuzz_db"
)

cursor = conn.cursor(buffered=True)  # 👈 important

cursor.execute("""
SELECT player_name, role
FROM players
JOIN teams ON players.team_id = teams.team_id
WHERE teams.country = 'India';
""")

for row in cursor:
    print(row)


('Virat Kohli', 'Batsman')
('Rohit Sharma', 'Batsman')
('Jasprit Bumrah', 'Bowler')


In [ ]:
import pandas as pd

cursor.execute("""
SELECT m.result, t1.team_name AS team1, t2.team_name AS team2,
       v.venue_name, v.city, m.match_date
FROM matches m
JOIN teams t1 ON m.team1_id = t1.team_id
JOIN teams t2 ON m.team2_id = t2.team_id
JOIN venues v ON m.venue_id = v.venue_id
WHERE m.match_date >= CURDATE() - INTERVAL 30 DAY
ORDER BY m.match_date DESC;
""")

rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]  # get column names

df = pd.DataFrame(rows, columns=columns)
print(df)


Empty DataFrame
Columns: [result, team1, team2, venue_name, city, match_date]
Index: []


In [ ]:
 cursor.execute("""
SELECT p.player_name, SUM(b.runs) AS total_runs
FROM batting_stats b
JOIN players p ON b.player_id = p.player_id
GROUP BY p.player_name
ORDER BY total_runs DESC
LIMIT 10;
""")

rows = cursor.fetchall()

 
for row in rows:
    print(row[0], int(row[1]))




Rohit Sharma 100
Virat Kohli 80


In [ ]:
cursor.execute("""
ALTER TABLE venues ADD COLUMN capacity INT;
""")
conn.commit()
print("✅ Capacity column added to venues table")


✅ Capacity column added to venues table


In [ ]:
cursor.execute("""
UPDATE venues SET capacity = 66000 WHERE venue_name = 'Eden Gardens';
""")
cursor.execute("""
UPDATE venues SET capacity = 60000 WHERE venue_name = 'MCG';
""")
cursor.execute("""
UPDATE venues SET capacity = 55000 WHERE venue_name = 'Wankhede Stadium';
""")
conn.commit()
print("✅ Venue capacities updated")


✅ Venue capacities updated


In [ ]:
cursor.execute("""
SELECT venue_name, city, country, capacity
FROM venues
WHERE capacity > 50000
ORDER BY capacity DESC;
""")

rows = cursor.fetchall()
for row in rows:
    print(row)


('MCG', 'Melbourne', 'Australia', 60000)
('Wankhede Stadium', 'Mumbai', 'India', 55000)


In [ ]:
cursor.execute("""
SELECT team_name, COUNT(*) AS wins
FROM matches
JOIN teams ON matches.result LIKE CONCAT('%', teams.team_name, '%')
GROUP BY team_name
ORDER BY wins DESC;
""")


In [ ]:
rows = cursor.fetchall()
for row in rows:
    print(row)


('India', 1)


In [ ]:
cursor.execute("""
SELECT role, COUNT(*) AS player_count
FROM players
GROUP BY role;
""")

for row in cursor.fetchall():
    print(row)


('Batsman', 3)
('Bowler', 2)


In [ ]:
cursor.execute("""
SELECT format, MAX(runs) AS highest_score
FROM batting_stats
GROUP BY format;
""")

for row in cursor.fetchall():
    print(row)


ProgrammingError: 1054 (42S22): Unknown column 'format' in 'field list'

In [ ]:
cursor.execute("SHOW COLUMNS FROM batting_stats;")
for row in cursor.fetchall():
    print(row)


('batting_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('match_id', 'int', 'YES', 'MUL', None, '')
('player_id', 'int', 'YES', 'MUL', None, '')
('runs', 'int', 'YES', '', None, '')
('balls', 'int', 'YES', '', None, '')
('fours', 'int', 'YES', '', None, '')
('sixes', 'int', 'YES', '', None, '')
('strike_rate', 'float', 'YES', '', None, '')


In [ ]:
cursor.execute("""
SELECT m.format, MAX(b.runs) AS highest_score
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
GROUP BY m.format;
""")

for row in cursor.fetchall():
    print(row)


ProgrammingError: 1054 (42S22): Unknown column 'm.format' in 'field list'

In [ ]:
cursor.execute("SHOW COLUMNS FROM matches;")
for row in cursor.fetchall():
    print(row)


('match_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('tournament_id', 'int', 'YES', 'MUL', None, '')
('team1_id', 'int', 'YES', 'MUL', None, '')
('team2_id', 'int', 'YES', 'MUL', None, '')
('venue_id', 'int', 'YES', 'MUL', None, '')
('match_date', 'date', 'YES', '', None, '')
('result', 'varchar(100)', 'YES', '', None, '')


In [ ]:
cursor.execute("SHOW COLUMNS FROM tournaments;")
for row in cursor.fetchall():
    print(row)


('tournament_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('name', 'varchar(100)', 'YES', '', None, '')
('year', 'int', 'YES', '', None, '')


In [ ]:
# Step 1: Add format column to tournaments
cursor.execute("""
ALTER TABLE tournaments ADD COLUMN format VARCHAR(20);
""")

# Step 2: Insert sample data into tournaments with format
cursor.execute("""
INSERT INTO tournaments (name, year, format)
VALUES
('ICC Cricket World Cup', 2023, 'ODI'),
('ICC T20 World Cup', 2022, 'T20'),
('ICC Champions Trophy', 2017, 'ODI'),
('ICC World Test Championship', 2021, 'Test');
""")

conn.commit()  # Save the changes

# Step 3: Select to check
cursor.execute("SELECT * FROM tournaments;")
for row in cursor.fetchall():
    print(row)

# Step 4: Query highest score by format
cursor.execute("""
SELECT t.format, MAX(b.runs) AS highest_score
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN tournaments t ON m.tournament_id = t.tournament_id
GROUP BY t.format;
""")

for row in cursor.fetchall():
    print(row)



ProgrammingError: 1060 (42S21): Duplicate column name 'format'

In [ ]:
# Step 1: Insert sample data into tournaments with format (skip ALTER since it's already done)
cursor.execute("""
INSERT INTO tournaments (name, year, format)
VALUES
('ICC Cricket World Cup', 2023, 'ODI'),
('ICC T20 World Cup', 2022, 'T20'),
('ICC Champions Trophy', 2017, 'ODI'),
('ICC World Test Championship', 2021, 'Test');
""")

# Step 2: Commit changes
conn.commit()

# Step 3: Select tournaments to verify
cursor.execute("SELECT * FROM tournaments;")
for row in cursor.fetchall():
    print(row)

# Step 4: Join batting_stats with matches and tournaments to find highest score by format
cursor.execute("""
SELECT t.format, MAX(b.runs) AS highest_score
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN tournaments t ON m.tournament_id = t.tournament_id
GROUP BY t.format;
""")
for row in cursor.fetchall():
    print(row)


(1, 'ICC World Cup', 2023, None)
(2, 'ICC Cricket World Cup', 2023, 'ODI')
(3, 'ICC T20 World Cup', 2022, 'T20')
(4, 'ICC Champions Trophy', 2017, 'ODI')
(5, 'ICC World Test Championship', 2021, 'Test')
(6, 'ICC Cricket World Cup', 2023, 'ODI')
(7, 'ICC T20 World Cup', 2022, 'T20')
(8, 'ICC Champions Trophy', 2017, 'ODI')
(9, 'ICC World Test Championship', 2021, 'Test')
(None, 100)


In [ ]:
cursor.execute("""
ALTER TABLE tournaments
ADD COLUMN host_country VARCHAR(50),
ADD COLUMN match_type VARCHAR(20),
ADD COLUMN start_date DATE,
ADD COLUMN total_matches INT;
""")


In [ ]:
cursor.execute("""
INSERT INTO tournaments (name, year, format, host_country, match_type, start_date, total_matches)
VALUES
('ICC T20 World Cup', 2024, 'T20', 'USA & West Indies', 'International', '2024-06-01', 55);
""")
conn.commit()


In [ ]:
cursor.execute("""
SELECT name, host_country, match_type, start_date, total_matches
FROM tournaments
WHERE YEAR(start_date) = 2024;
""")
for row in cursor.fetchall():
    print(row)


('ICC T20 World Cup', 'USA & West Indies', 'International', datetime.date(2024, 6, 1), 55)


In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS career_stats (
    player_id INT AUTO_INCREMENT PRIMARY KEY,
    player_name VARCHAR(100),
    role VARCHAR(50),
    total_runs INT,
    total_wickets INT,
    format VARCHAR(20)
);
""")



In [ ]:
cursor.execute("""
INSERT INTO career_stats (player_name, role, total_runs, total_wickets, format)
VALUES
('Virat Kohli', 'Batsman', 12000, 4, 'ODI'),
('Ben Stokes', 'All-rounder', 4500, 150, 'Test'),
('Hardik Pandya', 'All-rounder', 1800, 70, 'T20'),
('Ravindra Jadeja', 'All-rounder', 2500, 200, 'Test');
""")
conn.commit()


In [ ]:
cursor.execute("""
SELECT player_name, total_runs, total_wickets, format
FROM career_stats
WHERE total_runs > 1000 AND total_wickets > 50 AND role = 'All-rounder';
""")


In [ ]:
# Fetch all rows
results = cursor.fetchall()

# Print results
for row in results:
    print(row)

('Ben Stokes', 4500, 150, 'Test')
('Hardik Pandya', 1800, 70, 'T20')
('Ravindra Jadeja', 2500, 200, 'Test')
('Ben Stokes', 4500, 150, 'Test')
('Hardik Pandya', 1800, 70, 'T20')
('Ravindra Jadeja', 2500, 200, 'Test')


In [ ]:
# Execute the query
cursor.execute("""
SELECT m.result, t1.team_name AS team1, t2.team_name AS team2,
       v.venue_name, m.match_date
FROM matches m
JOIN teams t1 ON m.team1_id = t1.team_id
JOIN teams t2 ON m.team2_id = t2.team_id
JOIN venues v ON m.venue_id = v.venue_id
ORDER BY m.match_date DESC
LIMIT 20;
""")

# Fetch all rows
results = cursor.fetchall()

# Print column names
columns = [desc[0] for desc in cursor.description]
print(columns)

# Print rows
for row in results:
    print(row)


['result', 'team1', 'team2', 'venue_name', 'match_date']
('India won by 6 wickets', 'India', 'Australia', 'Wankhede Stadium', datetime.date(2023, 10, 15))


In [ ]:
# Execute query
cursor.execute("""
SELECT player_name, 
       SUM(CASE WHEN format='Test' THEN runs ELSE 0 END) AS test_runs,
       SUM(CASE WHEN format='ODI' THEN runs ELSE 0 END) AS odi_runs,
       SUM(CASE WHEN format='T20I' THEN runs ELSE 0 END) AS t20_runs,
       AVG(batting_average) AS overall_avg
FROM format_stats
GROUP BY player_name
HAVING COUNT(DISTINCT format) >= 2;
""")

# Fetch results
results = cursor.fetchall()

# Print column names
columns = [desc[0] for desc in cursor.description]
print(columns)

# Print rows
for row in results:
    print(row)


ProgrammingError: 1146 (42S02): Table 'cricbuzz_db.format_stats' doesn't exist

In [ ]:
cursor.execute("""
CREATE TABLE format_stats (
    player_id INT,
    player_name VARCHAR(100),
    format ENUM('Test','ODI','T20I'),
    matches INT,
    runs INT,
    batting_average FLOAT
);
""")


In [ ]:
cursor.executemany("""
INSERT INTO format_stats (player_id, player_name, format, matches, runs, batting_average)
VALUES (%s, %s, %s, %s, %s, %s)
""", [
    (1, "Virat Kohli", "ODI", 275, 12898, 57.3),
    (1, "Virat Kohli", "T20I", 115, 4008, 52.7),
    (1, "Virat Kohli", "Test", 111, 8676, 49.3),
    (2, "Rohit Sharma", "ODI", 245, 9825, 48.7),
    (2, "Rohit Sharma", "T20I", 148, 3853, 31.8),
    (2, "Rohit Sharma", "Test", 52, 3677, 46.5)
])
conn.commit()


In [ ]:
# Execute query
cursor.execute("""
SELECT player_name, 
       SUM(CASE WHEN format='Test' THEN runs ELSE 0 END) AS test_runs,
       SUM(CASE WHEN format='ODI' THEN runs ELSE 0 END) AS odi_runs,
       SUM(CASE WHEN format='T20I' THEN runs ELSE 0 END) AS t20_runs,
       AVG(batting_average) AS overall_avg
FROM format_stats
GROUP BY player_name
HAVING COUNT(DISTINCT format) >= 2;
""")

# Fetch results
results = cursor.fetchall()

# Print column names
columns = [desc[0] for desc in cursor.description]
print(columns)

# Print rows
for row in results:
    print(row)


['player_name', 'test_runs', 'odi_runs', 't20_runs', 'overall_avg']
('Rohit Sharma', Decimal('7354'), Decimal('19650'), Decimal('7706'), 42.333333333333336)
('Virat Kohli', Decimal('17352'), Decimal('25796'), Decimal('8016'), 53.09999974568685)


In [ ]:
cursor.execute("""
SELECT t.team_name,
       SUM(CASE WHEN v.country = t.country THEN 1 ELSE 0 END) AS home_matches,
       SUM(CASE WHEN v.country != t.country THEN 1 ELSE 0 END) AS away_matches
FROM matches m
JOIN teams t ON t.team_id IN (m.team1_id, m.team2_id)
JOIN venues v ON m.venue_id = v.venue_id
GROUP BY t.team_name;
""")


In [ ]:
cursor.execute("""
SELECT p1.player_name AS batsman1, p2.player_name AS batsman2,
       SUM(bp.runs) AS partnership_runs, bp.innings_id
FROM batting_partnerships bp
JOIN players p1 ON bp.player1_id = p1.player_id
JOIN players p2 ON bp.player2_id = p2.player_id
WHERE ABS(bp.position1 - bp.position2) = 1
GROUP BY bp.player1_id, bp.player2_id, bp.innings_id
HAVING partnership_runs >= 100;
""")


ProgrammingError: 1146 (42S02): Table 'cricbuzz_db.batting_partnerships' doesn't exist

In [ ]:
# ✅ Create batting_partnerships table with position1 and position2
cursor.execute("""
CREATE TABLE IF NOT EXISTS batting_partnerships (
    partnership_id INT AUTO_INCREMENT PRIMARY KEY,
    player1_id INT NOT NULL,
    player2_id INT NOT NULL,
    innings_id INT NOT NULL,
    position1 INT NOT NULL,
    position2 INT NOT NULL,
    runs INT NOT NULL,
    FOREIGN KEY (player1_id) REFERENCES players(player_id),
    FOREIGN KEY (player2_id) REFERENCES players(player_id)
);
""")
conn.commit()
print("✅ batting_partnerships table created")

# ✅ Insert some sample records (for testing)
cursor.executemany("""
INSERT INTO batting_partnerships (player1_id, player2_id, innings_id, position1, position2, runs)
VALUES (%s, %s, %s, %s, %s, %s);
""", [
    (1, 2, 101, 1, 2, 120),  # Opening partnership
    (3, 4, 101, 3, 4, 85),
    (5, 6, 102, 5, 6, 150),
    (7, 8, 103, 7, 9, 95)   # Not consecutive
])
db.commit()
print("✅ Sample partnerships inserted")

# ✅ Query consecutive batting partnerships with 100+ runs
cursor.execute("""
SELECT p1.player_name AS batsman1, p2.player_name AS batsman2,
       SUM(bp.runs) AS partnership_runs, bp.innings_id
FROM batting_partnerships bp
JOIN players p1 ON bp.player1_id = p1.player_id
JOIN players p2 ON bp.player2_id = p2.player_id
WHERE ABS(bp.position1 - bp.position2) = 1
GROUP BY bp.player1_id, bp.player2_id, bp.innings_id
HAVING partnership_runs >= 100;
""")

results = cursor.fetchall()

# ✅ Print results neatly
for row in results:
    print(f"{row[0]} & {row[1]} → {row[2]} runs (Innings {row[3]})")


ProgrammingError: 2055: Cursor is not connected

In [ ]:
print("hello")

hello


In [ ]:
cursor.execute("""
SELECT p.player_name, v.venue_name,
       AVG(b.economy) AS avg_economy,
       SUM(b.wickets) AS total_wickets,
       COUNT(*) AS matches_played
FROM bowling_stats b
JOIN players p ON b.player_id = p.player_id
JOIN matches m ON b.match_id = m.match_id
JOIN venues v ON m.venue_id = v.venue_id
WHERE b.overs >= 4
GROUP BY p.player_name, v.venue_name
HAVING matches_played >= 3;
""")


ProgrammingError: 2055: Cursor is not connected

In [ ]:
import mysql.connector

# Reconnect to your database
conn = mysql.connector.connect(
    host="localhost",       # change if needed
    user="root",            # your username
    password="monikha_19",# your password
    database="cricbuzz_db" # your database
)

# Create a new cursor
cursor = conn.cursor()


In [ ]:
# ✅ Create batting_partnerships table with position1 and position2
cursor.execute("""
CREATE TABLE IF NOT EXISTS batting_partnerships (
    partnership_id INT AUTO_INCREMENT PRIMARY KEY,
    player1_id INT NOT NULL,
    player2_id INT NOT NULL,
    innings_id INT NOT NULL,
    position1 INT NOT NULL,
    position2 INT NOT NULL,
    runs INT NOT NULL,
    FOREIGN KEY (player1_id) REFERENCES players(player_id),
    FOREIGN KEY (player2_id) REFERENCES players(player_id)
);
""")
conn.commit()
print("✅ batting_partnerships table created")

# ✅ Insert some sample records (for testing)
cursor.executemany("""
INSERT INTO batting_partnerships (player1_id, player2_id, innings_id, position1, position2, runs)
VALUES (%s, %s, %s, %s, %s, %s);
""", [
    (1, 2, 101, 1, 2, 120),  # Opening partnership
    (3, 4, 101, 3, 4, 85),
    (5, 6, 102, 5, 6, 150),
    (7, 8, 103, 7, 9, 95)   # Not consecutive
])
conn.commit()
print("✅ Sample partnerships inserted")

# ✅ Query consecutive batting partnerships with 100+ runs
cursor.execute("""
SELECT p1.player_name AS batsman1, p2.player_name AS batsman2,
       SUM(bp.runs) AS partnership_runs, bp.innings_id
FROM batting_partnerships bp
JOIN players p1 ON bp.player1_id = p1.player_id
JOIN players p2 ON bp.player2_id = p2.player_id
WHERE ABS(bp.position1 - bp.position2) = 1
GROUP BY bp.player1_id, bp.player2_id, bp.innings_id
HAVING partnership_runs >= 100;
""")

results = cursor.fetchall()

# ✅ Print results neatly
for row in results:
    print(f"{row[0]} & {row[1]} → {row[2]} runs (Innings {row[3]})")

✅ batting_partnerships table created


IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cricbuzz_db`.`batting_partnerships`, CONSTRAINT `batting_partnerships_ibfk_3` FOREIGN KEY (`innings_id`) REFERENCES `innings` (`innings_id`))

In [ ]:
cursor.execute("DROP TABLE IF EXISTS batting_partnerships;")
conn.commit()

cursor.execute("""
CREATE TABLE batting_partnerships (
    partnership_id INT AUTO_INCREMENT PRIMARY KEY,
    player1_id INT NOT NULL,
    player2_id INT NOT NULL,
    innings_id INT NOT NULL,
    position1 INT NOT NULL,
    position2 INT NOT NULL,
    runs INT NOT NULL,
    FOREIGN KEY (player1_id) REFERENCES players(player_id),
    FOREIGN KEY (player2_id) REFERENCES players(player_id),
    FOREIGN KEY (innings_id) REFERENCES innings(innings_id)
)
""")
conn.commit()


In [ ]:
# ✅ Insert some sample records (for testing)
cursor.executemany("""
INSERT INTO batting_partnerships (player1_id, player2_id, innings_id, position1, position2, runs)
VALUES (%s, %s, %s, %s, %s, %s);
""", [
    (1, 2, 101, 1, 2, 120),  # Opening partnership
    (3, 4, 101, 3, 4, 85),
    (5, 6, 102, 5, 6, 150),
    (7, 8, 103, 7, 9, 95)   # Not consecutive
])
conn.commit()
print("✅ Sample partnerships inserted")

# ✅ Query consecutive batting partnerships with 100+ runs
cursor.execute("""
SELECT p1.player_name AS batsman1, p2.player_name AS batsman2,
       SUM(bp.runs) AS partnership_runs, bp.innings_id
FROM batting_partnerships bp
JOIN players p1 ON bp.player1_id = p1.player_id
JOIN players p2 ON bp.player2_id = p2.player_id
WHERE ABS(bp.position1 - bp.position2) = 1
GROUP BY bp.player1_id, bp.player2_id, bp.innings_id
HAVING partnership_runs >= 100;
""")

results = cursor.fetchall()

# ✅ Print results neatly
for row in results:
    print(f"{row[0]} & {row[1]} → {row[2]} runs (Innings {row[3]})")

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cricbuzz_db`.`batting_partnerships`, CONSTRAINT `batting_partnerships_ibfk_3` FOREIGN KEY (`innings_id`) REFERENCES `innings` (`innings_id`))

In [ ]:
# Example: Check innings table
cursor.execute("SELECT * FROM innings;")
for row in cursor.fetchall():
    print(row)

# Example: Check players table
cursor.execute("SELECT * FROM players;")
for row in cursor.fetchall():
    print(row)


(1, 1, 2, 250, 10, 48.3)
(2, 1, 1, 251, 4, 47.2)
(1, 'Virat Kohli', 'Batsman', 1)
(2, 'Rohit Sharma', 'Batsman', 1)
(3, 'Jasprit Bumrah', 'Bowler', 1)
(4, 'Steve Smith', 'Batsman', 2)
(5, 'Pat Cummins', 'Bowler', 2)


In [ ]:
# Sample batting partnerships
cursor.executemany("""
INSERT INTO batting_partnerships (player1_id, player2_id, innings_id, position1, position2, runs)
VALUES (%s, %s, %s, %s, %s, %s);
""", [
    (1, 2, 1, 1, 2, 120),  # Virat & Rohit in innings 1
    (4, 5, 2, 1, 2, 95),   # Steve & Pat in innings 2
])
conn.commit()


In [ ]:
# ✅ Insert some sample records (for testing)
cursor.executemany("""
INSERT INTO batting_partnerships (player1_id, player2_id, innings_id, position1, position2, runs)
VALUES (%s, %s, %s, %s, %s, %s);
""", [
    (1, 2, 101, 1, 2, 120),  # Opening partnership
    (3, 4, 101, 3, 4, 85),
    (5, 6, 102, 5, 6, 150),
    (7, 8, 103, 7, 9, 95)   # Not consecutive
])
conn.commit()
print("✅ Sample partnerships inserted")

# ✅ Query consecutive batting partnerships with 100+ runs
cursor.execute("""
SELECT p1.player_name AS batsman1, p2.player_name AS batsman2,
       SUM(bp.runs) AS partnership_runs, bp.innings_id
FROM batting_partnerships bp
JOIN players p1 ON bp.player1_id = p1.player_id
JOIN players p2 ON bp.player2_id = p2.player_id
WHERE ABS(bp.position1 - bp.position2) = 1
GROUP BY bp.player1_id, bp.player2_id, bp.innings_id
HAVING partnership_runs >= 100;
""")

results = cursor.fetchall()

# ✅ Print results neatly
for row in results:
    print(f"{row[0]} & {row[1]} → {row[2]} runs (Innings {row[3]})")

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cricbuzz_db`.`batting_partnerships`, CONSTRAINT `batting_partnerships_ibfk_3` FOREIGN KEY (`innings_id`) REFERENCES `innings` (`innings_id`))

In [ ]:
cursor.execute("SELECT * FROM innings;")
for row in cursor.fetchall():
    print(row)


(1, 1, 2, 250, 10, 48.3)
(2, 1, 1, 251, 4, 47.2)


In [ ]:
cursor.execute("SELECT * FROM players;")
for row in cursor.fetchall():
    print(row)


(1, 'Virat Kohli', 'Batsman', 1)
(2, 'Rohit Sharma', 'Batsman', 1)
(3, 'Jasprit Bumrah', 'Bowler', 1)
(4, 'Steve Smith', 'Batsman', 2)
(5, 'Pat Cummins', 'Bowler', 2)


In [ ]:
cursor.executemany("""
INSERT INTO batting_partnerships (player1_id, player2_id, innings_id, position1, position2, runs)
VALUES (%s, %s, %s, %s, %s, %s);
""", [
    (1, 2, 1, 1, 2, 120),  # Virat & Rohit in innings 1
    (4, 5, 2, 1, 2, 95),   # Steve & Pat in innings 2
])
conn.commit()


In [ ]:
cursor.execute("""
SELECT p1.player_name AS batsman1, p2.player_name AS batsman2,
       SUM(bp.runs) AS partnership_runs, bp.innings_id
FROM batting_partnerships bp
JOIN players p1 ON bp.player1_id = p1.player_id
JOIN players p2 ON bp.player2_id = p2.player_id
WHERE ABS(bp.position1 - bp.position2) = 1
GROUP BY bp.player1_id, bp.player2_id, bp.innings_id
HAVING partnership_runs >= 100;
""")

for row in cursor.fetchall():
    print(row)


('Virat Kohli', 'Rohit Sharma', Decimal('240'), 1)
('Steve Smith', 'Pat Cummins', Decimal('190'), 2)


In [ ]:
for row in cursor.fetchall():
    batsman1, batsman2, partnership_runs, innings_id = row
    print(batsman1, batsman2, int(partnership_runs), innings_id)


In [ ]:
cursor.execute("""
SELECT p1.player_name AS batsman1, p2.player_name AS batsman2,
       SUM(bp.runs) AS partnership_runs, bp.innings_id
FROM batting_partnerships bp
JOIN players p1 ON bp.player1_id = p1.player_id
JOIN players p2 ON bp.player2_id = p2.player_id
WHERE ABS(bp.position1 - bp.position2) = 1
GROUP BY bp.player1_id, bp.player2_id, bp.innings_id
HAVING partnership_runs >= 100;
""")
rows = cursor.fetchall()  # fetch once
print("Number of rows:", len(rows))  # check if any rows exist

for row in rows:
    batsman1, batsman2, partnership_runs, innings_id = row
    print(batsman1, batsman2, int(partnership_runs), innings_id)


Number of rows: 2
Virat Kohli Rohit Sharma 240 1
Steve Smith Pat Cummins 190 2


In [ ]:
# Execute the query
cursor.execute("""
SELECT p.player_name, v.venue_name,
       AVG(b.economy) AS avg_economy,
       SUM(b.wickets) AS total_wickets,
       COUNT(*) AS matches_played
FROM bowling_stats b
JOIN players p ON b.player_id = p.player_id
JOIN matches m ON b.match_id = m.match_id
JOIN venues v ON m.venue_id = v.venue_id
WHERE b.overs >= 4
GROUP BY p.player_name, v.venue_name
HAVING matches_played >= 3;
""")

# Fetch all rows
rows = cursor.fetchall()

# Print nicely, converting AVG to float
for row in rows:
    player_name, venue_name, avg_economy, total_wickets, matches_played = row
    print(
        player_name,
        venue_name,
        round(float(avg_economy), 2),  # round average economy to 2 decimals
        total_wickets,
        matches_played
    )


In [ ]:
cursor.execute("SELECT * FROM bowling_stats LIMIT 5;")
for row in cursor.fetchall():
    print(row)


(1, 1, 3, 10.0, 1, 40, 3, 4.0)
(2, 1, 5, 9.2, 0, 55, 2, 5.9)


In [ ]:
cursor.execute("""
SELECT p.player_name, v.venue_name,
       AVG(b.economy) AS avg_economy,
       SUM(b.wickets) AS total_wickets,
       COUNT(*) AS matches_played
FROM bowling_stats b
JOIN players p ON b.player_id = p.player_id
JOIN matches m ON b.match_id = m.match_id
JOIN venues v ON m.venue_id = v.venue_id
WHERE b.overs >= 4
GROUP BY p.player_name, v.venue_name
HAVING matches_played >= 1;  -- reduced from 3
""")

for row in cursor.fetchall():
    player_name, venue_name, avg_economy, total_wickets, matches_played = row
    print(player_name, venue_name, round(float(avg_economy),2), total_wickets, matches_played)


Jasprit Bumrah Wankhede Stadium 4.0 3 1
Pat Cummins Wankhede Stadium 5.9 2 1


In [ ]:
# Execute the query
cursor.execute("""
SELECT p.player_name,
       AVG(b.runs) AS avg_runs,
       COUNT(*) AS close_matches_played
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE (m.result LIKE '%wickets%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 5)
   OR (m.result LIKE '%runs%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 50)
GROUP BY p.player_name;
""")

# Fetch all rows
rows = cursor.fetchall()

# Print neatly
for row in rows:
    player_name, avg_runs, close_matches_played = row
    print(player_name, round(float(avg_runs), 2), close_matches_played)


DatabaseError: 2014 (HY000): Commands out of sync; you can't run this command now

In [ ]:
 

# Discard any pending results
try:
    cursor.fetchall()
except:
    pass



In [ ]:
cursor.execute("""
SELECT p.player_name,
       AVG(b.runs) AS avg_runs,
       COUNT(*) AS close_matches_played
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE (m.result LIKE '%wickets%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 5)
   OR (m.result LIKE '%runs%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 50)
GROUP BY p.player_name;
""")
rows = cursor.fetchall()
for row in rows:
    player_name, avg_runs, close_matches_played = row
    print(player_name, round(float(avg_runs), 2), close_matches_played)


DatabaseError: 2014 (HY000): Commands out of sync; you can't run this command now

In [ ]:
# Make sure you have a buffered cursor
cursor = conn.cursor(buffered=True)


OperationalError: MySQL Connection not available.

In [ ]:
print(conn.is_connected())  # Should print True


False


In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="monikha_19",
    database="cricbuzz_db"
)

# Create a buffered cursor
cursor = conn.cursor(buffered=True)


In [ ]:
cursor.execute("""
SELECT p.player_name,
       AVG(b.runs) AS avg_runs,
       COUNT(*) AS close_matches_played
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE (m.result LIKE '%wickets%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 5)
   OR (m.result LIKE '%runs%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 50)
GROUP BY p.player_name;
""")

rows = cursor.fetchall()
for row in rows:
    player_name, avg_runs, close_matches_played = row
    print(player_name, round(float(avg_runs),2), close_matches_played)


In [ ]:
cursor.execute("SELECT match_id, result FROM matches LIMIT 10;")
for row in cursor.fetchall():
    print(row)


(1, 'India won by 6 wickets')


In [ ]:
cursor.execute("""
SELECT p.player_name,
       AVG(b.runs) AS avg_runs,
       COUNT(*) AS close_matches_played
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE (m.result LIKE '%wickets%' AND CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(m.result, ' ', -2), ' ', 1) AS UNSIGNED) < 5)
   OR (m.result LIKE '%runs%' AND CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(m.result, ' ', -2), ' ', 1) AS UNSIGNED) < 50)
GROUP BY p.player_name;
""")

rows = cursor.fetchall()
for row in rows:
    player_name, avg_runs, close_matches_played = row
    print(player_name, round(float(avg_runs), 2), close_matches_played)


In [ ]:
cursor.execute("""
SELECT p.player_name,
       AVG(b.runs) AS avg_runs,
       COUNT(*) AS close_matches_played,
       m.result,
       CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(m.result, ' ', -2), ' ', 1) AS UNSIGNED) AS margin
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE m.result LIKE '%wickets%' OR m.result LIKE '%runs%'
GROUP BY p.player_name, m.result
LIMIT 20;
""")

rows = cursor.fetchall()
for row in rows:
    print(row)


('Virat Kohli', Decimal('80.0000'), 1, 'India won by 6 wickets', 6)
('Rohit Sharma', Decimal('100.0000'), 1, 'India won by 6 wickets', 6)


In [ ]:
 
for row in rows:
    player_name, avg_runs, close_matches_played, _, _ = row  # ignore result and margin
    print(player_name, round(float(avg_runs), 2), close_matches_played)


Virat Kohli 80.0 1
Rohit Sharma 100.0 1


In [ ]:
# Execute the query
cursor.execute("""
SELECT p.player_name, YEAR(m.match_date) AS year,
       AVG(b.runs) AS avg_runs, AVG(b.strike_rate) AS avg_sr, COUNT(*) AS matches_played
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE YEAR(m.match_date) >= 2020
GROUP BY p.player_name, year
HAVING matches_played >= 5;
""")

# Fetch all rows
rows = cursor.fetchall()

# Print results neatly
for row in rows:
    player_name, year, avg_runs, avg_sr, matches_played = row
    print(
        player_name,
        year,
        round(float(avg_runs), 2),   # round average runs to 2 decimals
        round(float(avg_sr), 2),     # round average strike rate to 2 decimals
        matches_played
    )


In [ ]:
cursor.execute("""
SELECT toss_decision,
       COUNT(*) AS total_matches,
       SUM(CASE WHEN result LIKE CONCAT('%', toss_winner, '%') THEN 1 ELSE 0 END) AS wins_by_toss_winner,
       ROUND(SUM(CASE WHEN result LIKE CONCAT('%', toss_winner, '%') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS win_percentage
FROM matches
GROUP BY toss_decision;
""")


ProgrammingError: 1054 (42S22): Unknown column 'toss_decision' in 'field list'

In [ ]:
# Add toss_winner, toss_decision, and format columns to matches
cursor.execute("""
ALTER TABLE matches
ADD COLUMN toss_winner INT,
ADD COLUMN toss_decision VARCHAR(10),
ADD COLUMN format VARCHAR(10),
ADD FOREIGN KEY (toss_winner) REFERENCES teams(team_id)
""")
print("✅ Updated 'matches' table with toss and format columns")


In [ ]:
print("hello")

hello


In [ ]:
cursor = conn.cursor(buffered=True)


NameError: name 'conn' is not defined

In [ ]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="monikha_19",
    database="cricbuzz_db"
)

# Create a buffered cursor
cursor = conn.cursor(buffered=True)

In [ ]:
# Add toss_winner, toss_decision, and format columns to matches
cursor.execute("""
ALTER TABLE matches
ADD COLUMN toss_winner INT,
ADD COLUMN toss_decision VARCHAR(10),
ADD COLUMN format VARCHAR(10),
ADD FOREIGN KEY (toss_winner) REFERENCES teams(team_id)
""")
print("✅ Updated 'matches' table with toss and format columns")


ProgrammingError: 1060 (42S21): Duplicate column name 'toss_winner'

In [ ]:
# Create player_stats table
cursor.execute("""
CREATE TABLE IF NOT EXISTS player_stats (
    stat_id INT AUTO_INCREMENT PRIMARY KEY,
    player_id INT,
    format VARCHAR(10),
    runs_scored INT,
    batting_average FLOAT,
    strike_rate FLOAT,
    wickets_taken INT,
    bowling_average FLOAT,
    economy_rate FLOAT,
    FOREIGN KEY (player_id) REFERENCES players(player_id)
)
""")
print("✅ Created 'player_stats' table")


✅ Created 'player_stats' table


In [ ]:
# Create batting_partnerships table
cursor.execute("""
CREATE TABLE IF NOT EXISTS batting_partnerships (
    partnership_id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    innings_id INT,
    player1_id INT,
    player2_id INT,
    position1 INT,
    position2 INT,
    runs INT,
    FOREIGN KEY (match_id) REFERENCES matches(match_id),
    FOREIGN KEY (innings_id) REFERENCES innings(innings_id),
    FOREIGN KEY (player1_id) REFERENCES players(player_id),
    FOREIGN KEY (player2_id) REFERENCES players(player_id)
)
""")
print("✅ Created 'batting_partnerships' table")


✅ Created 'batting_partnerships' table


In [ ]:
cursor.execute("SHOW COLUMNS FROM matches")
for col in cursor.fetchall():
    print(col)


('match_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('tournament_id', 'int', 'YES', 'MUL', None, '')
('team1_id', 'int', 'YES', 'MUL', None, '')
('team2_id', 'int', 'YES', 'MUL', None, '')
('venue_id', 'int', 'YES', 'MUL', None, '')
('match_date', 'date', 'YES', '', None, '')
('result', 'varchar(100)', 'YES', '', None, '')
('toss_winner', 'int', 'YES', 'MUL', None, '')
('toss_decision', 'varchar(10)', 'YES', '', None, '')
('format', 'varchar(10)', 'YES', '', None, '')


In [ ]:
INSERT INTO matches (tournament_id, team1_id, team2_id, venue_id, match_date, result, toss_winner, toss_decision, format)
VALUES (1, 101, 102, 201, '2023-05-15', 'Team 101 won by 5 wickets', 101, 'bat', 'T20');


SyntaxError: invalid syntax (3508620056.py, line 1)

In [ ]:
# Insert a match record
insert_query = """
INSERT INTO matches (tournament_id, team1_id, team2_id, venue_id, match_date, result, toss_winner, toss_decision, format)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data = (1, 101, 102, 201, '2023-05-15', 'Team 101 won by 5 wickets', 101, 'bat', 'T20')

cursor.execute(insert_query, data)
conn.commit()

print("✅ Match record inserted successfully!")


IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cricbuzz_db`.`matches`, CONSTRAINT `matches_ibfk_2` FOREIGN KEY (`team1_id`) REFERENCES `teams` (`team_id`))

In [ ]:
team_insert_query = """
INSERT INTO teams (team_id, team_name)
VALUES (%s, %s)
"""

teams_data = [
    (101, 'Team Alpha'),
    (102, 'Team Beta')
]

cursor.executemany(team_insert_query, teams_data)
conn.commit()

print("✅ Teams inserted successfully!")


✅ Teams inserted successfully!


In [ ]:
# Insert a match record
insert_query = """
INSERT INTO matches (tournament_id, team1_id, team2_id, venue_id, match_date, result, toss_winner, toss_decision, format)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data = (1, 101, 102, 201, '2023-05-15', 'Team 101 won by 5 wickets', 101, 'bat', 'T20')

cursor.execute(insert_query, data)
conn.commit()

print("✅ Match record inserted successfully!")

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cricbuzz_db`.`matches`, CONSTRAINT `matches_ibfk_4` FOREIGN KEY (`venue_id`) REFERENCES `venues` (`venue_id`))

In [ ]:
venue_insert_query = """
INSERT INTO venues (venue_id, venue_name, city)
VALUES (%s, %s, %s)
"""

venue_data = (201, 'M.A. Chidambaram Stadium', 'Chennai')

cursor.execute(venue_insert_query, venue_data)
conn.commit()

print("✅ Venue inserted successfully!")


✅ Venue inserted successfully!


In [ ]:
insert_query = """
INSERT INTO matches (tournament_id, team1_id, team2_id, venue_id, match_date, result, toss_winner, toss_decision, format)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data = (1, 101, 102, 201, '2023-05-15', 'Team 101 won by 5 wickets', 101, 'bat', 'T20')

cursor.execute(insert_query, data)
conn.commit()

print("✅ Match record inserted successfully!")


✅ Match record inserted successfully!


In [ ]:
player_stats_query = """
INSERT INTO player_stats (match_id, player_id, team_id, runs_scored, balls_faced, wickets_taken)
VALUES (%s, %s, %s, %s, %s, %s)
"""

player_stats_data = [
    (1, 1001, 101, 45, 30, 0),
    (1, 1002, 101, 10, 8, 2),
    (1, 1003, 102, 60, 40, 1),
    (1, 1004, 102, 5, 12, 3)
]

cursor.executemany(player_stats_query, player_stats_data)
conn.commit()

print("✅ Sample player stats inserted!")


ProgrammingError: 1054 (42S22): Unknown column 'match_id' in 'field list'

In [ ]:
cursor.execute("""
ALTER TABLE player_stats
ADD COLUMN match_id INT,
ADD FOREIGN KEY (match_id) REFERENCES matches(match_id)
""")
conn.commit()

print("✅ match_id column added to player_stats!")


✅ match_id column added to player_stats!


In [ ]:
player_stats_query = """
INSERT INTO player_stats (match_id, player_id, team_id, runs_scored, balls_faced, wickets_taken)
VALUES (%s, %s, %s, %s, %s, %s)
"""

player_stats_data = [
    (1, 1001, 101, 45, 30, 0),
    (1, 1002, 101, 10, 8, 2),
    (1, 1003, 102, 60, 40, 1),
    (1, 1004, 102, 5, 12, 3)
]

cursor.executemany(player_stats_query, player_stats_data)
conn.commit()

print("✅ Sample player stats inserted!")

ProgrammingError: 1054 (42S22): Unknown column 'team_id' in 'field list'

In [ ]:
cursor.execute("""
ALTER TABLE player_stats
ADD COLUMN team_id INT,
ADD FOREIGN KEY (team_id) REFERENCES teams(team_id)
""")
conn.commit()

print("✅ team_id column added to player_stats!")


✅ team_id column added to player_stats!


In [ ]:
player_stats_query = """
INSERT INTO player_stats (match_id, player_id, team_id, runs_scored, balls_faced, wickets_taken)
VALUES (%s, %s, %s, %s, %s, %s)
"""

player_stats_data = [
    (1, 1001, 101, 45, 30, 0),
    (1, 1002, 101, 10, 8, 2),
    (1, 1003, 102, 60, 40, 1),
    (1, 1004, 102, 5, 12, 3)
]

cursor.executemany(player_stats_query, player_stats_data)
conn.commit()

print("✅ Sample player stats inserted!")

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cricbuzz_db`.`player_stats`, CONSTRAINT `player_stats_ibfk_1` FOREIGN KEY (`player_id`) REFERENCES `players` (`player_id`))

In [ ]:
cursor.execute("""
ALTER TABLE player_stats
ADD COLUMN balls_faced INT
""")
conn.commit()

print("✅ balls_faced column added to player_stats!")


✅ balls_faced column added to player_stats!


In [ ]:
player_insert_query = """
INSERT INTO players (player_id, player_name, role)
VALUES (%s, %s, %s)
"""

players_data = [
    (1001, 'Rohit Sharma', 'Batsman'),
    (1002, 'Hardik Pandya', 'All-Rounder'),
    (1003, 'Virat Kohli', 'Batsman'),
    (1004, 'Jasprit Bumrah', 'Bowler')
]

cursor.executemany(player_insert_query, players_data)
conn.commit()

print("✅ Sample players inserted!")


✅ Sample players inserted!


In [ ]:
player_stats_query = """
INSERT INTO player_stats (match_id, player_id, team_id, runs_scored, balls_faced, wickets_taken)
VALUES (%s, %s, %s, %s, %s, %s)
"""

player_stats_data = [
    (1, 1001, 101, 45, 30, 0),
    (1, 1002, 101, 10, 8, 2),
    (1, 1003, 102, 60, 40, 1),
    (1, 1004, 102, 5, 12, 3)
]

cursor.executemany(player_stats_query, player_stats_data)
conn.commit()

print("✅ Sample player stats inserted!")


✅ Sample player stats inserted!


In [ ]:
partnership_query = """
INSERT INTO batting_partnerships (match_id, striker_id, non_striker_id, runs, balls)
VALUES (%s, %s, %s, %s, %s)
"""

partnership_data = [
    (1, 1001, 1002, 55, 40),
    (1, 1003, 1004, 20, 25)
]

cursor.executemany(partnership_query, partnership_data)
conn.commit()

print("✅ Sample batting partnerships inserted!")


ProgrammingError: 1054 (42S22): Unknown column 'match_id' in 'field list'

In [ ]:
cursor.execute("SHOW COLUMNS FROM batting_partnerships")
for col in cursor.fetchall():
    print(col)


('partnership_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('player1_id', 'int', 'NO', 'MUL', None, '')
('player2_id', 'int', 'NO', 'MUL', None, '')
('innings_id', 'int', 'NO', 'MUL', None, '')
('position1', 'int', 'NO', '', None, '')
('position2', 'int', 'NO', '', None, '')
('runs', 'int', 'NO', '', None, '')


In [ ]:
partnership_query = """
INSERT INTO batting_partnerships (player1_id, player2_id, innings_id, position1, position2, runs)
VALUES (%s, %s, %s, %s, %s, %s)
"""

partnership_data = [
    (1001, 1002, 1, 1, 2, 55),
    (1003, 1004, 1, 3, 4, 20)
]

cursor.executemany(partnership_query, partnership_data)
conn.commit()

print("✅ Sample batting partnerships inserted!")


✅ Sample batting partnerships inserted!


In [ ]:
cursor.execute("""
SELECT p.player_name,
       AVG(b.runs) AS avg_runs,
       COUNT(*) AS close_matches_played
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE m.result LIKE '%wickets%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 5
   OR m.result LIKE '%runs%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 50
GROUP BY p.player_name;
""")


In [ ]:
# Step 3: Execute your query
query = """
SELECT p.player_name,
       AVG(b.runs) AS avg_runs,
       COUNT(*) AS close_matches_played
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE (m.result LIKE '%wickets%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 5)
   OR (m.result LIKE '%runs%' AND CAST(SUBSTRING_INDEX(m.result, ' ', -2) AS UNSIGNED) < 50)
GROUP BY p.player_name;
"""

cursor.execute(query)

# Step 4: Fetch results
results = cursor.fetchall()  # fetchall() gets all rows

# Step 5: Print results
for row in results:
    print(row)

In [ ]:
# Step 3: Write your query
query = """
SELECT p.player_name, YEAR(m.match_date) AS year,
       AVG(b.runs) AS avg_runs, AVG(b.strike_rate) AS avg_sr, COUNT(*) AS matches_played
FROM batting_stats b
JOIN matches m ON b.match_id = m.match_id
JOIN players p ON b.player_id = p.player_id
WHERE YEAR(m.match_date) >= 2020
GROUP BY p.player_name, year
HAVING matches_played >= 5;
"""

# Step 4: Execute the query
cursor.execute(query)

# Step 5: Fetch all results
results = cursor.fetchall()  # gets all rows as a list of tuples

# Step 6: Print results neatly
for row in results:
    print(f"Player: {row[0]}, Year: {row[1]}, Avg Runs: {row[2]:.2f}, Avg SR: {row[3]:.2f}, Matches: {row[4]}")

In [ ]:
# Step 3: Write your query
query = """
SELECT toss_decision,
       COUNT(*) AS total_matches,
       SUM(CASE WHEN result LIKE CONCAT('%', toss_winner, '%') THEN 1 ELSE 0 END) AS wins_by_toss_winner,
       ROUND(SUM(CASE WHEN result LIKE CONCAT('%', toss_winner, '%') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS win_percentage
FROM matches
GROUP BY toss_decision;
"""

# Step 4: Execute the query
cursor.execute(query)

In [ ]:
cursor.execute("""
SELECT p.player_name,
       AVG(b.economy) AS avg_economy,
       SUM(b.wickets) AS total_wickets,
       COUNT(*) AS matches_played
FROM bowling_stats b
JOIN players p ON b.player_id = p.player_id
JOIN matches m ON b.match_id = m.match_id
WHERE format IN ('ODI', 'T20') AND b.overs >= 2
GROUP BY p.player_name
HAVING matches_played >= 10;
""")


In [ ]:
cursor.execute("""
SELECT p.player_name,
       AVG(b.runs) AS avg_runs,
       STDDEV(b.runs) AS run_stddev
FROM batting_stats b
JOIN players p ON b.player_id = p.player_id
JOIN matches m ON b.match_id = m.match_id
WHERE b.balls >= 10 AND YEAR(m.match_date) >= 2022
GROUP BY p.player_name;
""")


In [ ]:
cursor.execute("""
SELECT player_name,
       SUM(CASE WHEN format='Test' THEN 1 ELSE 0 END) AS test_matches,
       SUM(CASE WHEN format='ODI' THEN 1 ELSE 0 END) AS odi_matches,
       SUM(CASE WHEN format='T20' THEN 1 ELSE 0 END) AS t20_matches,
       AVG(batting_average) AS overall_avg
FROM format_stats
GROUP BY player_name;
""")

results = cursor.fetchall()

for row in results:
    print(row)


('Virat Kohli', Decimal('2'), Decimal('2'), Decimal('0'), 53.09999974568685)
('Rohit Sharma', Decimal('2'), Decimal('2'), Decimal('0'), 42.333333333333336)


In [ ]:
cursor.execute("""
SELECT player_name, format,
       ROUND((runs_scored * 0.01) + (batting_average * 0.5) + (strike_rate * 0.3), 2) AS batting_points,
       ROUND((wickets_taken * 2) + ((50 - bowling_average) * 0.5) + ((6 - economy_rate) * 2), 2) AS bowling_points,
       ROUND(((runs_scored * 0.01) + (batting_average * 0.5) + (strike_rate * 0.3)) +
             ((wickets_taken * 2) + ((50 - bowling_average) * 0.5) + ((6 - economy_rate) * 2)), 2) AS total_score
FROM player_stats
ORDER BY total_score DESC;
""")


ProgrammingError: 1054 (42S22): Unknown column 'player_name' in 'field list'

In [ ]:
cursor.execute("""
SELECT t1.team_name AS team1, t2.team_name AS team2,
       COUNT(*) AS total_matches,
       SUM(CASE WHEN m.result LIKE CONCAT('%', t1.team_name, '%') THEN 1 ELSE 0 END) AS team1_wins,
       SUM(CASE WHEN m.result LIKE CONCAT('%', t2.team_name, '%') THEN 1 ELSE 0 END) AS team2_wins,
       ROUND(SUM(CASE WHEN m.result LIKE CONCAT('%', t1.team_name, '%') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS team1_win_pct,
       ROUND(SUM(CASE WHEN m.result LIKE CONCAT('%', t2.team_name, '%') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS team2_win_pct
FROM matches m
JOIN teams t1 ON m.team1_id = t1.team_id
JOIN teams t2 ON m.team2_id = t2.team_id
WHERE m.match_date >= CURDATE() - INTERVAL 3 YEAR
GROUP BY team1, team2
HAVING total_matches >= 5;
""")


In [ ]:
cursor.execute("""
SELECT player_name,
       AVG(CASE WHEN rn <= 5 THEN runs ELSE NULL END) AS avg_last_5,
       AVG(CASE WHEN rn <= 10 THEN runs ELSE NULL END) AS avg_last_10,
       SUM(CASE WHEN runs >= 50 AND rn <= 10 THEN 1 ELSE 0 END) AS fifties_last_10,
       STDDEV(CASE WHEN rn <= 10 THEN runs ELSE NULL END) AS consistency_score
FROM (
    SELECT b.player_id, p.player_name, b.runs,
           ROW_NUMBER() OVER (PARTITION BY b.player_id ORDER BY m.match_date DESC) AS rn
    FROM batting_stats b
    JOIN players p ON b.player_id = p.player_id
    JOIN matches m ON b.match_id = m.match_id
) recent
GROUP BY player_name;
""")


In [ ]:
cursor.execute("""
SELECT p1.player_name AS batsman1, p2.player_name AS batsman2,
       AVG(bp.runs) AS avg_partnership,
       SUM(CASE WHEN bp.runs >= 50 THEN 1 ELSE 0 END) AS fifty_plus_partnerships,
       MAX(bp.runs) AS highest_partnership,
       ROUND(SUM(CASE WHEN bp.runs >= 50 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS success_rate
FROM batting_partnerships bp
JOIN players p1 ON bp.player1_id = p1.player_id
JOIN players p2 ON bp.player2_id = p2.player_id
WHERE ABS(bp.position1 - bp.position2) = 1
GROUP BY bp.player1_id, bp.player2_id
HAVING COUNT(*) >= 5
ORDER BY success_rate DESC;
""")


In [ ]:
cursor.execute("""
SELECT player_name, year, quarter,
       AVG(runs) AS avg_runs, AVG(strike_rate) AS avg_sr,
       LAG(AVG(runs)) OVER (PARTITION BY player_name ORDER BY year, quarter) AS prev_avg_runs,
       CASE
           WHEN AVG(runs) > LAG(AVG(runs)) OVER (PARTITION BY player_name ORDER BY year, quarter) THEN 'Improving'
           WHEN AVG(runs) < LAG(AVG(runs)) OVER (PARTITION BY player_name ORDER BY year, quarter) THEN 'Declining'
           ELSE 'Stable'
       END AS performance_trend
FROM (
    SELECT p.player_name, b.runs, b.strike_rate,
           YEAR(m.match_date) AS year,
           QUARTER(m.match_date) AS quarter
    FROM batting_stats b
    JOIN players p ON b.player_id = p.player_id
    JOIN matches m ON b.match_id = m.match_id
) stats
GROUP BY player_name, year, quarter
HAVING COUNT(*) >= 3;
""")


In [ ]:
#21
cursor.execute("SHOW COLUMNS FROM player_stats")
for col in cursor.fetchall():
    print(col)


('stat_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('player_id', 'int', 'YES', 'MUL', None, '')
('format', 'varchar(10)', 'YES', '', None, '')
('runs_scored', 'int', 'YES', '', None, '')
('batting_average', 'float', 'YES', '', None, '')
('strike_rate', 'float', 'YES', '', None, '')
('wickets_taken', 'int', 'YES', '', None, '')
('bowling_average', 'float', 'YES', '', None, '')
('economy_rate', 'float', 'YES', '', None, '')
('match_id', 'int', 'YES', 'MUL', None, '')
('team_id', 'int', 'YES', 'MUL', None, '')
('balls_faced', 'int', 'YES', '', None, '')


In [ ]:
cursor.execute("""
SELECT p.player_name, ps.format,
       ROUND((ps.runs_scored * 0.01) + (ps.batting_average * 0.5) + (ps.strike_rate * 0.3), 2) AS batting_points,
       ROUND((ps.wickets_taken * 2) + ((50 - ps.bowling_average) * 0.5) + ((6 - ps.economy_rate) * 2), 2) AS bowling_points,
       ROUND(
           ((ps.runs_scored * 0.01) + (ps.batting_average * 0.5) + (ps.strike_rate * 0.3)) +
           ((ps.wickets_taken * 2) + ((50 - ps.bowling_average) * 0.5) + ((6 - ps.economy_rate) * 2)), 2
       ) AS total_score
FROM player_stats ps
JOIN players p ON ps.player_id = p.player_id
ORDER BY total_score DESC;
""")
